In [41]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
from tqdm import tqdm
import time
import random
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow_probability import distributions as tfd


# Load Data and utility functions 

In [2]:
## season 2 dataframes 
train_x_2 = pd.read_csv('train_x_df.csv')
train_y_2 = pd.read_csv('train_y_df.csv') 
test_x_2 = pd.read_csv('test_x_df.csv') 
submission = pd.read_csv('sample_submission.csv') 

train_x_2.shape, train_y_2.shape, test_x_2.shape, submission.shape


((10572180, 12), (919320, 12), (738300, 12), (535, 3))

In [3]:
def df2d_to_array3d(df_2d):
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d


x_train = df2d_to_array3d(train_x_2) 
y_train = df2d_to_array3d(train_y_2) 
x_test = df2d_to_array3d(test_x_2) 

x_train.shape, y_train.shape, x_test.shape


((7661, 1380, 10), (7661, 120, 10), (535, 1380, 10))

In [4]:
def plot_series(x_series, y_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'actual_series') 
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [5]:
def plot_predicted_series(x_series, y_predicted):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_predicted)),
             y_predicted, label = 'predicted_series') 
    #plt.axhline(1, c = 'red')
    plt.legend()


In [6]:
x_train_close = x_train[:,:,4] 
y_train_close = y_train[:,:,4] 
x_test_close = x_test[:,:,4] 

close_prices = np.concatenate([x_train_close, y_train_close], axis = 1) 
close_prices.shape

(7661, 1500)

In [7]:
eps = 1e-8

close_prices = np.log(close_prices + eps) 


In [8]:
x_test_close = np.log(x_test_close + eps) 

# Preprocess Data
Given time series data (t_1, t_2, ..., tN) predict t{N+K} Here, we let K = 120 and N is a hyperparameter, but we can let it be 60 minutes.

In [9]:
K = 120 
N = 60 
seq_len = 1500 
features = 1
X = [] 
Y = [] 

for j in tqdm(range(close_prices.shape[0]), position = 0, leave = True): 
    i = 0
    while i+N+K < 1500: 
        X.append(close_prices[j, i:i+N]) 
        Y.append(close_prices[j, i+N+K]) 
        i += 1   
        
        
X = np.asarray(X) 
Y = np.asarray(Y)

X.shape, Y.shape

100%|██████████| 7661/7661 [00:09<00:00, 783.14it/s]


((10112520, 60), (10112520,))

In [10]:
X = X.reshape((-1,N,features)) 
Y = Y.reshape((-1,features))
X.shape, Y.shape

((10112520, 60, 1), (10112520, 1))

# Define Model

In [48]:
no_parameters = 3
components = 4

def nnelu(input):
    """ Computes the Non-Negative Exponential Linear Unit
    """
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def slice_parameter_vectors(parameter_vector):
    """ Returns an unpacked list of paramter vectors.
    """
    return [parameter_vector[:,i*components:(i+1)*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    """ Computes the mean negative log-likelihood loss of y given the mixture parameters.
    """
    alpha, mu, sigma = slice_parameter_vectors(parameter_vector) # Unpack parameter vectors
    
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Normal(
            loc=mu,       
            scale=sigma))
    
    log_likelihood = gm.log_prob(tf.transpose(y)) # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1)

tf.keras.utils.get_custom_objects().update({'nnelu': Activation(nnelu)})

In [35]:
def transformer_block(inputs, node, drop_rate, activation): 
    attn_output = MultiHeadAttention(num_heads = 2, key_dim = node)(inputs, inputs) 
    attn_output = Dropout(drop_rate)(attn_output) 
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output) 
    ffn_output = Dense(node, activation = activation)(out1) 
    ffn_output = Dense(node)(ffn_output) 
    ffn_output = Dropout(drop_rate)(ffn_output)
    out2 = LayerNormalization(epsilon=1e-6)(out1 + ffn_output) 
    return out2
    
    
def build_transformer(node = 64, activation = 'relu', drop_rate = 0.2, num_layers = 3):  
    inputs = Input((N, features)) 
    bn = BatchNormalization()(inputs)
    x = Conv1D(node*2, 5, activation = activation)(bn) 
    x = MaxPooling1D(3)(x) 
    x = Dropout(drop_rate)(x) 
    x = Conv1D(node, 5, activation = activation)(x) 
    x = MaxPooling1D(3)(x) 
    x = Dropout(drop_rate)(x) 
    
    positions = tf.range(start=0, limit=x.shape[1], delta=1)
    positions = Embedding(input_dim = x.shape[1], output_dim = node)(positions) 
    x = x + positions 
    
    x = transformer_block(x, node, drop_rate, activation) 
        
    x = GlobalMaxPooling1D()(x)
    x = Dropout(drop_rate)(x)  
    
    
    alpha_v = Dense(components, activation = 'softmax')(x)
    mu_v = Dense(components)(x) 
    sigma_v = Dense(components, activation = 'nnelu')(x)
    
    outputs = Concatenate()([alpha_v, mu_v, sigma_v])
    
    model = Model(inputs=inputs,outputs=outputs) 
    model.compile(loss = gnll_loss, optimizer = 'adam') 
    return model


In [36]:
model = build_transformer() 
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 60, 1)]      0                                            
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 60, 1)        4           input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 56, 128)      768         batch_normalization_4[0][0]      
__________________________________________________________________________________________________
max_pooling1d_8 (MaxPooling1D)  (None, 18, 128)      0           conv1d_8[0][0]                   
____________________________________________________________________________________________

In [37]:
model_path = 'Transformer_MDN_Close_epoch_{epoch:03d}_val_{val_loss:.3f}.h5'
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 1, factor = 0.5)
checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10) 


history = model.fit(x=X, 
                    y=Y, 
                    batch_size = 32, 
                    epochs = 100, 
                    callbacks = [learning_rate_reduction, checkpoint, early_stopping], 
                    validation_split = 0.1)


Epoch 1/100
284415/284415 [==============================] - 1887s 7ms/step - loss: -2.7836 - val_loss: -3.0229

Epoch 00001: val_loss improved from inf to -3.02289, saving model to Transformer_MDN_Close_epoch_001_val_-3.023.h5
Epoch 2/100
117579/284415 [===========>..................] - ETA: 17:27 - loss: -2.8877

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



284415/284415 [==============================] - 1871s 7ms/step - loss: -2.8881 - val_loss: -2.9714

Epoch 00002: val_loss did not improve from -3.02289
Epoch 3/100
284415/284415 [==============================] - 1866s 7ms/step - loss: -2.8889 - val_loss: -2.9103

Epoch 00003: val_loss did not improve from -3.02289
Epoch 4/100
284415/284415 [==============================] - 1843s 6ms/step - loss: -2.8923 - val_loss: -2.9252

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00004: val_loss did not improve from -3.02289
Epoch 5/100
284415/284415 [==============================] - 1825s 6ms/step - loss: -2.9119 - val_loss: -2.9422

Epoch 00005: val_loss did not improve from -3.02289
Epoch 6/100
284415/284415 [==============================] - 1815s 6ms/step - loss: -2.9119 - val_loss: -2.9125

Epoch 00006: val_loss did not improve from -3.02289
Epoch 7/100
284415/284415 [==============================] - 1823s 6ms/step - loss: -2.9136 - val_loss: -2

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



278871/284415 [============================>.] - ETA: 33s - loss: -2.9226

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



193107/284415 [===================>..........] - ETA: 9:16 - loss: -2.9286

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



265615/284415 [===========================>..] - ETA: 1:54 - loss: -2.9287

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Make Prediction

In [54]:
best_model = load_model('Transformer_MDN_Close_epoch_001_val_-3.023.h5', custom_objects={'Activation':Activation(nnelu), 'gnll_loss':gnll_loss})

best_model.summary() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 60, 1)]      0                                            
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 60, 1)        4           input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 56, 128)      768         batch_normalization_4[0][0]      
__________________________________________________________________________________________________
max_pooling1d_8 (MaxPooling1D)  (None, 18, 128)      0           conv1d_8[0][0]                   
____________________________________________________________________________________________

In [55]:
## We need to preprocess inputs for prediction 
X_test = [] 
for j in tqdm(range(x_test_close.shape[0]), position = 0, leave = True): 
    for i in range(seq_len-N-K-120, seq_len-N-K):
        X_test.append(x_test_close[j, i:i+N])  

X_test = np.asarray(X_test).reshape((-1,N,features))

X_test.shape


100%|██████████| 535/535 [00:00<00:00, 16534.79it/s]


(64200, 60, 1)

In [59]:
predicted = best_model.predict(X_test) 

predicted = predicted.reshape((-1,12,120)) 
predicted.shape

(535, 12, 120)

In [60]:
alpha, mu, sigma = slice_parameter_vectors(predicted)

In [61]:
alpha.shape, mu.shape, sigma.shape

((535, 4, 120), (535, 4, 120), (535, 4, 120))

In [63]:
## decide buy quantity and sell time 
## first get the predicted prices (i.e. mean of each distribution)
sell_prices = [] 
for i in tqdm(range(alpha.shape[0]), position = 0, leave = True): 
    sample_prices = [] 
    for j in range(120): 
        price = 0 
        for k in range(components):
            price += alpha[i,k,j] * mu[i,k,j]
        sample_prices.append(price) 
    sell_prices.append(sample_prices) 
    
sell_prices = np.asarray(sell_prices) 
sell_prices.shape

100%|██████████| 535/535 [00:00<00:00, 1567.65it/s]


(535, 120)

In [80]:
## second, compute the standard deviation of each distribution 
## note that the mus are stored inside the array sell_prices 
stdevs = [] 
for i in tqdm(range(alpha.shape[0]), position = 0, leave = True): 
    sample_stdevs = [] 
    for j in range(120): 
        stdev = 0 
        for k in range(components): 
            stdev += alpha[i,k,j] * (sigma[i,k,j]*sigma[i,k,j] + mu[i,k,j]*mu[i,k,j]) 
        stdev = stdev - (sell_prices[i,j] * sell_prices[i,j])  
        sample_stdevs.append(stdev)
    stdevs.append(sample_stdevs) 

stdevs = np.asarray(stdevs) 
stdevs.shape

100%|██████████| 535/535 [00:00<00:00, 828.19it/s]


(535, 120)

In [83]:
## Select the time when the highest mean occurs  
## Use Kelly Criterion to decide buy quantity 

buy_quantities = [] 
sell_times = [] 

for i in tqdm(range(sell_prices.shape[0]), position = 0, leave = True): 
    sell_time = np.argmax(sell_prices[i,:]) 
    sell_times.append(sell_time) 
    
    buy_price = x_test_close[i,-1]  
    returns = sell_prices[i,sell_time] - buy_price   
    buy_quantity = returns / (stdevs[i,sell_time] * stdevs[i,sell_time])  
    
    buy_quantities.append(buy_quantity)  
    
buy_quantities = np.asarray(buy_quantities) 
sell_times = np.asarray(sell_times)


100%|██████████| 535/535 [00:00<00:00, 125648.28it/s]


In [ ]:
# min-max normalize buy_quantities 
scaler = MinMaxScaler() 


In [74]:
buy_quantities.shape, sell_times.shape

((535,), (535,))